In [1]:
!pip3 install pandas tqdm

     |████████████████████████████████| 12.2 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 8.9 MB/s  eta 0:00:01
     |████████████████████████████████| 499 kB 14.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# import

In [2]:
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from glob import glob

# Load csv

In [3]:
label_df = pd.read_csv('./dataset/original/trainLabels.csv')
label_df

,Id,Class
0,01kcPWA9K2BOxQeS5Rju,1
1,04EjIdbPV5e1XroFOpiN,1
2,05EeG39MTRrI6VY21DPd,1
3,05rJTUWYAKNegBk2wE8X,1
4,0AnoOZDNbPXIr2MRBSCJ,1
...,...,...
10863,KFrZ0Lop1WDGwUtkusCi,9
10864,kg24YRJTB8DNdKMXpwOH,9
10865,kG29BLiFYPgWtpb350sO,9
10866,kGITL4OJxYMWEQ1bKBiP,9


# Create TFRecord

가장 작은 바이너리 크기 = 172032

256xn이 가장 작은 이미지 크기

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
def tfwriter(label_df):
    writer = tf.io.TFRecordWriter('./dataset/train.tfrecord')
    for idx, row in tqdm(label_df.iterrows()):
        img_path = f"./dataset/img/img256/{row['Id']}.png"
        malware_class = row['Class']
        
        img_string = tf.io.read_file(img_path)
        tf_example = image_example(img_string, malware_class)
        writer.write(tf_example.SerializeToString())

In [ ]:
tfwriter(label_df)